# Density Learner

This code implements the hierarchical model described in Yan Karklin's thesis:<br>
Y Karklin (2007) - <i>Hierarchical Statistical Models of Computations in the Visual Cortex</i>

#### Notes for training the model (copied with occasional modifications from Yan's thesis):

  * When $\mathbf{v}=0$ the variances of all the linear coefficients are $1$, and the joint density reduces to the standard i.i.d. form of the standard linear models
  * In theory, the number of variance coeffients can be greater or smaller than the number of linear coefficients, although Yan typically has many fewer
  * Emperically, the outputs of linear filters are well fit by a Laplacian or the slightly more sparse generalized Gaussian with $q=0.7$ for the generalized Gaussian equation $p(x) \propto e^{-|x|^{q}}$
    * For the coefficient likelihood function, I used the Laplacian, where $q=1.0$
  * A symmetric prior on $\mathbf{v}$ implies that the learned variance patterns are symmetric - a pattern of high and low variances is as likely as its converse, low and high. Alternatively, we can restrict $\mathbf{v}$ to be all-positive, dropping this assumption, but this can be computationally tricky when gradient methods for MAP estimation are employed.
  * Adding weak weight decay or fixing the norm of the vectors in $\mathbf{A}$ and $\mathbf{B}$ helps alleviate degenerate conditions introduced by the approximations of marginalization over the latent variables.
  * They used the diagonal terms of the Hessian to stabilize and speed up the inference procedure by adjusting the step size along each dimension of $\mathbf{v}$.
  * The matrices $\mathbf{A}$ and $\mathbf{B}$ can be optimized concurrently by interleaving gradient ascent steps on $\mathbf{A}$ and $\mathbf{B}$.
  * In most of their simulations they manually adjusted the (presumably $l_{2}$) norm of the parameters to maintin the desired level of variance for the latent variables.
  * They applied learning to 20x20 image patches sampled from 40 images of outdoor scenes (Doi et al., 2003), although I used the Van Hateren natural scene dataset.
  * Image preprocessing:
     * Low-pass radially symmetric filter to eliminate high frequencie
     * DC component was removed (0 center)
     * Whitened by premultiplying with $\mathbf{C}^{-\frac{1}{2}}$, where $\mathbf{C}$ is the data covariance matrix
  * They used a complete set of linear basis functions (400)
  * The number of variance components was set to 100
  * The noise variance $\sigma^{2}_{\epsilon}$ was set to $0.1$
  * All basis functions were initialized to small (presumably uniformly distributed) random values
  * A batch size of 300 patches was used for training
  * The algorithm was run for 10,000 iterations
  * A step size of 0.1 (tapered for the last 1,000 iterations) was used to learn $\mathbf{A}$
  * The step size for adapting $\mathbf{B}$ was gradually increased at the beginning of the simulation because emergence of the variance patterns requires some stabilization in the basis functions in $\mathbf{A}$

#### Equations:

* The approximation to the log-likelihood function is given by:<br>
$\hat{L}=-\text{log}|\text{det}\mathbf{A}|+\sum_{i}\text{log}p(s_{i}|B,\hat{\mathbf{v}})+\sum_{j}\text{log}p(\hat{v}_{j})$<br>
$\hat{L}\propto-\text{log}|\text{det}\mathbf{A}|+\sum_{i}\left(-\frac{[\mathbf{B}\hat{\mathbf{v}}]_{i}}{2}-\frac{\sqrt{2}|s_{i}|}{e^{[\mathbf{B}\hat{\mathbf{v}}]_{i}/2}}\right)-\sum_{j}|v_{j}|$

* The update for the variance coefficients is given by:<br>
$\hat{v}^{new}_{j} \leftarrow \hat{v}^{old}_{j} + \epsilon_{v} \left(\sum_{i}B_{ij}\left(|\bar{s}_{i}|-1\right) - \phi^{'}_{v}(\hat{v}^{old}_{j})\right)$

* The Hessian of the negative log-likelihood is:<br>
$\frac{\partial^{2}\hat{L}}{\partial v_{j} \partial v_{k}}=\frac{1}{4}\sum_{i}B_{ij}B_{ik}\frac{\sqrt{2}|s_{i}|}{e^{[\mathbf{B}\mathbf{v}]_{i}/2}}$<br>
$\frac{\partial^{2}\hat{L}}{\partial\mathbf{v}\partial\mathbf{v}}=\frac{1}{4}\mathbf{B}^{T}\mathbf{\bar{S}}\mathbf{B}$<br>
where $\mathbf{\bar{S}}$ is a diagonal matrix containing the variance-normalized coefficient magnitudes, $\mathbf{\bar{S}}=\text{daig}\left(\left|\bar{s}_{1}\right|,\left|\bar{s}_{2}\right|,\ldots,\left|\bar{s}_{I}\right|\right)$.

* In the noiseless complete case, $s=A^{-1}x$, they optimized the inverse of the linear basis functions, the filter $\mathbf{W}$, using the natural gradient:<br>
$\frac{\partial\hat{L}}{\partial\mathbf{W}}=\left(\mathbf{I}+\phi^{'}(\mathbf{s})\mathbf{s}^{T}\right)\mathbf{W}$
 

* In the noisy case, they used the sparse coding update rule:<br>
$\frac{\partial\hat{L}}{\partial\mathbf{A}}=\left(\mathbf{x}-\mathbf{A}\mathbf{\hat{s}}\right)\mathbf{s^{T}}$

* The higher-order parameters in B are obtained by following the gradient:<br>
$\frac{\partial\hat{L}}{\partial B_{ij}}=-v_{j}+v_{j}\frac{\sqrt{2}|s_{i}|}{e^{[\mathbf{B}\mathbf{v}]_{i}/2}}$